In [1]:
cd ../backend/macrobenchmark/

/Users/dorislee/Desktop/Research/lux/lux-benchmark/backend/macrobenchmark


In [177]:
import glob
import time
import papermill as pm
import pandas as pd
import numpy as np
import json
count = []
with open("airbnb/airbnb.ipynb") as json_file:
    data = json.load(json_file)
    for cell in data['cells']:
        label = cell["source"][0]
        cell_type = None
        if "# {{NO LUX}}" in label:
            # Exclude the first two non-lux cell (all imports or read csv)
            # code = ''.join(cell["source"])
            # if "pd.read_csv" not in code and "import pandas as pd" not in code:
            cell_type = "python"
        elif "# {{PRINT SERIES}}" in label:
            cell_type = "print_series"
        elif "# {{PRINT DF}}" in label:
            cell_type = "print_dataframe"
        count.append(cell_type)
count = pd.Series(count)
print(count.value_counts())

python             17
print_dataframe    14
print_series        7
dtype: int64


In [178]:
trial = []

for nb_name in glob.glob("airbnb/airbnb_*_output.ipynb"):
# for nb_name in glob.glob("airbnb/airbnb_*_*_output.ipynb"):
    snb = nb_name.split("_")
    if len(snb)==4:
        _, condition, nPts, _ = nb_name.split("_")
    elif len(snb)==3:
        condition = "pandas"
        _,  nPts, _ = nb_name.split("_")
    with open(nb_name) as json_file:
        data = json.load(json_file)
        for cell in data['cells']:
            label = cell["source"][0]
            cell_type = None
            if "# {{NO LUX}}" in label:
                cell_type = "python"
            elif "# {{PRINT SERIES}}" in label:
                cell_type = "print_series"
            elif "# {{PRINT DF}}" in label:
                cell_type = "print_dataframe"
            duration = cell["metadata"]["papermill"]["duration"]
            trial.append([condition, nPts,cell_type, duration])
#             from datetime import datetime
#             if ("execution" in cell["metadata"]):
#                 starttime = datetime.strptime(cell["metadata"]["execution"]["iopub.execute_input"],"%Y-%m-%dT%H:%M:%S.%fZ")
#                 endtime = datetime.strptime(cell["metadata"]["execution"]["shell.execute_reply"],"%Y-%m-%dT%H:%M:%S.%fZ")
#                 duration = (endtime-starttime).total_seconds()
#     #             print(nb_name, cell_type, duration)
#                 trial.append([condition, nPts,cell_type, duration])
# #             else:
#                 print ("skip:",cell)

In [179]:
trial = pd.DataFrame(trial,columns=["condition","nPts","cell_type","time"])

In [180]:
trial.to_csv("../../paper/airbnb_macrobenchmark.csv",index=None)

In [181]:
# trial.groupby("cell_type").count()

--------------

In [182]:
import pandas as pd
import altair as alt

In [183]:
overall = trial.groupby("condition").sum().reset_index()

In [184]:
alt.Chart(overall).mark_bar().encode(
    x = "condition",
    y = alt.Y("time",scale=alt.Scale(type='log'),title="log(total time)"),
)

alt.Chart(...)

In [185]:
overall = trial.groupby(["condition","nPts"]).sum().reset_index()

In [186]:
# overall=overall[overall["time"]!=0]

In [187]:
chart = alt.Chart(overall,title = "Overall").mark_bar().encode(
    x = "condition",
    y = alt.Y("time",scale=alt.Scale(type='log'),title="log(total time)"),
    column="nPts",
    color="condition",
    tooltip=["time"]
)
chart = chart.configure_title(fontSize=16, offset=5, orient='top', anchor='middle')
chart

alt.Chart(...)

In [188]:
overall = trial.groupby(["condition","nPts","cell_type"]).mean().reset_index()
pdf = overall[overall["cell_type"] =="python"]

In [189]:
a = trial[(trial["nPts"]=="10000")&(trial["condition"]=="o1o2")&(trial["cell_type"]=="print_dataframe")][["nPts","cell_type","time"]]

In [190]:
b = trial[(trial["nPts"]=="10000")&(trial["condition"]=="o1")&(trial["cell_type"]=="print_dataframe")][["nPts","cell_type","time"]].rename(columns={"time":"o1time"})

In [191]:
# ab = a.merge(b,on=["nPts","cell_type"])

# ab["diff"] = ab["o1time"]-ab["time"]

In [192]:
overall = trial.groupby(["condition","nPts","cell_type"]).mean().reset_index()
pdf = overall[overall["cell_type"] =="python"]
chart = alt.Chart(pdf,title = "Non-Lux Python Operations").mark_bar().encode(
    x = "condition",
    y = alt.Y("time",scale=alt.Scale(type='log'),title="log(total time)"),
    column="nPts",
    color="condition",
    tooltip=["time"]
)
chart = chart.configure_title(fontSize=16, offset=5, orient='top', anchor='middle')
chart


alt.Chart(...)

In [206]:
trial[(trial["cell_type"] =="python")&(trial["condition"]!="baseline")].groupby("nPts")

,condition,nPts,cell_type,time
44,o1o2o3,1000000,python,0.345245
47,o1o2o3,1000000,python,25.980364
48,o1o2o3,1000000,python,0.026671
50,o1o2o3,1000000,python,0.203193
54,o1o2o3,1000000,python,0.029457
...,...,...,...,...
822,o1,10000,python,0.036500
826,o1,10000,python,0.036173
831,o1,10000,python,0.039830
833,o1,10000,python,0.049041


In [15]:
overall = trial.groupby(["condition","nPts","cell_type"]).mean().reset_index()
pdf = overall[overall["cell_type"] =="print_series"]
chart = alt.Chart(pdf,title = "Print Series").mark_bar().encode(
    x = "condition",
    y = alt.Y("time",scale=alt.Scale(type='log'),title="log(total time)"),
    column="nPts",
    color="condition",
    tooltip=["time"]
)
chart = chart.configure_title(fontSize=16, offset=5, orient='top', anchor='middle')
chart

alt.Chart(...)

In [16]:
overall = trial.groupby(["condition","nPts","cell_type"]).mean().reset_index()
pdf = overall[overall["cell_type"] =="print_dataframe"]
chart = alt.Chart(pdf,title = "Print Dataframe").mark_bar().encode(
    x = "condition",
    y = alt.Y("time",scale=alt.Scale(type='log'),title="log(total time)"),
    column="nPts",
    color="condition",
    tooltip=["time"]
)
chart = chart.configure_title(fontSize=16, offset=5, orient='top', anchor='middle')
chart

alt.Chart(...)

------------

### Communities post processing

In [217]:
count = []
with open("communities/communities.ipynb") as json_file:
    data = json.load(json_file)
    for cell in data['cells']:
        label = cell["source"][0]
        cell_type = None
        CELL_STOP = 45
        if ("execution_count" in cell and cell["execution_count"] and int(cell["execution_count"])<CELL_STOP):
            if "# {{NO LUX}}" in label:
                cell_type = "python"
            elif "# {{PRINT SERIES}}" in label:
                cell_type = "print_series"
            elif "# {{PRINT DF}}" in label:
                cell_type = "print_dataframe"
        count.append(cell_type)
count = pd.Series(count)
print(count.value_counts())

python             25
print_dataframe    14
print_series        4
dtype: int64


In [139]:
import glob
import time
import papermill as pm
import pandas as pd
import numpy as np
import json
trial = []

for nb_name in glob.glob("communities/communities_*_output.ipynb"):
# for nb_name in glob.glob("airbnb/airbnb_*_*_output.ipynb"):
    snb = nb_name.split("_")
    if len(snb)==4:
        _, condition, nPts, _ = nb_name.split("_")
        CELL_STOP = 46
    elif len(snb)==3:
        condition = "pandas"
        _,  nPts, _ = nb_name.split("_")
        CELL_STOP = 45
    with open(nb_name) as json_file:
        data = json.load(json_file)
        for cell in data['cells']:
            # Keep only cell count <46 (last few cells long runnning)
            if ("execution_count" in cell and cell["execution_count"] and int(cell["execution_count"])<CELL_STOP):
                label = cell["source"][0]
                cell_type = None
                if "# {{NO LUX}}" in label:
                    cell_type = "python"
                elif "# {{PRINT SERIES}}" in label:
                    cell_type = "print_series"
                elif "# {{PRINT DF}}" in label:
                    cell_type = "print_dataframe"
                duration = cell["metadata"]["papermill"]["duration"]
    #             print(nb_name, cell_type, duration)
                trial.append([condition, nPts,cell_type, duration])
    #             duration = cell["metadata"]["papermill"]["duration"]
    #             from datetime import datetime
    #             if ("execution" in cell["metadata"]):
    #                 starttime = datetime.strptime(cell["metadata"]["execution"]["iopub.execute_input"],"%Y-%m-%dT%H:%M:%S.%fZ")
    #                 endtime = datetime.strptime(cell["metadata"]["execution"]["shell.execute_reply"],"%Y-%m-%dT%H:%M:%S.%fZ")
    #                 duration = (endtime-starttime).total_seconds()
    #     #             print(nb_name, cell_type, duration)
    #                 trial.append([condition, nPts,cell_type, duration])
    # #             else:
    #                 print ("skip:",cell)

trial = pd.DataFrame(trial,columns=["condition","nPts","cell_type","time"])

In [140]:
trial.to_csv("../../paper/communities_macrobenchmark.csv",index=None)

In [142]:
overall = trial.groupby(["condition","nPts"]).sum().reset_index()
overall=overall[overall["time"]!=0]
chart = alt.Chart(overall,title = "Overall").mark_bar().encode(
    x = "condition",
    y = alt.Y("time",scale=alt.Scale(type='log'),title="log(total time)"),
    column="nPts",
    color="condition",
    tooltip=["time"]
)
chart = chart.configure_title(fontSize=16, offset=5, orient='top', anchor='middle')
chart

alt.Chart(...)

In [143]:
overall = trial.groupby(["condition","nPts","cell_type"]).mean().reset_index()
pdf = overall[overall["cell_type"] =="python"]
chart = alt.Chart(pdf,title = "Non-Lux Python Operations").mark_bar().encode(
    x = "condition",
    y = alt.Y("time",scale=alt.Scale(type='log'),title="log(total time)"),
    column="nPts",
    color="condition",
    tooltip=["time"]
)
chart = chart.configure_title(fontSize=16, offset=5, orient='top', anchor='middle')
chart

alt.Chart(...)

In [145]:
overall = trial.groupby(["condition","nPts","cell_type"]).mean().reset_index()
pdf = overall[overall["cell_type"] =="print_series"]
chart = alt.Chart(pdf,title = "Print Series").mark_bar().encode(
    x = "condition",
    y = alt.Y("time",scale=alt.Scale(type='log'),title="log(total time)"),
    column="nPts",
    color="condition",
    tooltip=["time"]
)
chart = chart.configure_title(fontSize=16, offset=5, orient='top', anchor='middle')
chart.interactive()

alt.Chart(...)

In [146]:
# trial[(trial["cell_type"] =="print_series")&(trial["condition"] =="o1o2")&((trial["nPts"] =="10000")|(trial["nPts"] =="100000"))]

In [147]:
# trial[(trial["cell_type"] =="print_series")&(trial["condition"] =="o1o2")&((trial["nPts"] =="10000")|(trial["nPts"] =="100000"))]

There are only 4 print series cells, which means that there is high variance in the execution time. If we do a rerun of experiments, we should add in more series cells. Esp since the metadata is recomputed for some of the series cells. Otherwise, the runtime is more or less the same. If we use papermill's "duration" v.s. subtract "execution" time (IPython messaging statrt/end), we get different results. So the conditions are more or less the same, but require reruns for the numbers to stable down.

In [176]:
# trial[(trial["cell_type"] =="print_dataframe")&(trial["condition"] =="o1o2")&(trial["nPts"] =="10000")]

In [149]:
overall = trial.groupby(["condition","nPts","cell_type"]).mean().reset_index()
pdf = overall[overall["cell_type"] =="print_dataframe"]
chart = alt.Chart(pdf,title = "Print Dataframe").mark_bar().encode(
    x = "condition",
    y = alt.Y("time",scale=alt.Scale(type='log'),title="log(total time)"),
    column="nPts",
    color="condition",
    tooltip=["time"]
)
chart = chart.configure_title(fontSize=16, offset=5, orient='top', anchor='middle')
chart

alt.Chart(...)

----------------

In [150]:
# # VegaLite can not do logarithmic stacked bar since it is misleading
# alt.Chart(df).mark_bar().encode(
#     x = alt.X("nPts",type="ordinal"),
#     y = alt.Y("time"),#,scale=alt.Scale(type='log'),title="log(total time)"),
#     color="condition"
# )

In [151]:
import numpy as np

In [218]:
def pctChange(s1,s2):
    return (np.array(s1["time"])-np.array(s2["time"]))/np.array(s2["time"])
def rawChange(s1,s2):
    return (np.array(s1["time"])-np.array(s2["time"]))

In [248]:
df = pd.read_csv("../../paper/airbnb_macrobenchmark.csv")

In [249]:
df= df.groupby(["condition","nPts"]).sum().reset_index()

In [250]:
pdonly = df[df["condition"]=="pandas"]
best = df[df["condition"]=="o1o2o3"]
worst = df[df["condition"]=="baseline"]

Intro: Naive implementation of recommendation on top of dataframes can be extremely costly incurring up to X% slowdown

In [251]:
pctChange(worst,best)

array([ 3.09677655,  5.30572715, 13.31293887, 18.75098951])

Abstract: We demonstrate that through the use of a careful design and three system optimizations, Lux incurs an overhead of no more than X% more time than pandas for a 10M row dataframe. 

In [228]:
pctChange(best,pdonly)
# rawChange(best,pdonly)

array([-0.06341819,  0.14775279,  0.82921703,  3.72622968])

In [220]:
df = df[df["cell_type"]=="print_dataframe"]
df = df.groupby(["nPts","condition"]).mean().reset_index()

In [221]:
# df = df.groupby(["nPts","condition"]).sum().reset_index()
pdonly = df[df["condition"]=="pandas"]
best = df[df["condition"]=="o1o2o3"]
worst = df[df["condition"]=="baseline"]

In [222]:
# pctChange(best,pdonly)

# pctChange(worst,pdonly)

rawChange(best,pdonly)

array([ 0.13209493,  0.37043429,  1.99063757, 21.18188143])

When the dataframe contains fewer than 1M rows for Airbnb, each print incurs no more than 2 seconds for each print in addition to \pandas (in the 10M case, each print incurred an overhead of 21 seconds).

In [161]:
pctChange(worst,best)

array([10.86215629,  9.99948866,  8.55949505,  7.05569083])

In [157]:
def singlePctChange(v1,v2):
    return (v1-v2)/v2

In [252]:
df = pd.read_csv("../../paper/communities_macrobenchmark.csv")

In [253]:
df= df.groupby(["condition","nPts"]).sum().reset_index()

In [254]:
# df = df.groupby(["nPts","condition"]).sum().reset_index()
pdonly = df[df["condition"]=="pandas"]
best = df[df["condition"]=="o1o2o3"]
worst = df[df["condition"]=="baseline"]

In [255]:
pctChange(worst,best)

array([ 88.80738415,  88.62974581, 575.01099458, 385.58727696])

In [165]:
df = df[df["cell_type"]=="print_dataframe"]
df = df.groupby(["nPts","condition"]).mean().reset_index()

In [166]:
pdonly = df[df["condition"]=="pandas"]
best = df[df["condition"]=="o1o2o3"]
worst = df[df["condition"]=="baseline"]

In [169]:
pctChange(worst,best)

array([ 53.59101284,  53.76013845, 345.7942322 , 226.01165928])

In [170]:
rawChange(best,pdonly)

array([0.96940779, 0.92198357, 0.81182914, 1.41636907])

 Likewise, for Communities, the overhead was no more than 1.2 seconds. 

In [174]:
df = pd.read_csv("../../paper/airbnb_macrobenchmark.csv")
df = df[df["cell_type"]=="print_series"]
df = df.groupby(["nPts","condition"]).mean().reset_index()

pdonly = df[df["condition"]=="pandas"]
o1 = df[df["condition"]=="o1"]
o1o2 = df[df["condition"]=="o1o2"]
best = df[df["condition"]=="o1o2o3"]
worst = df[df["condition"]=="baseline"]

overhead = rawChange(best,pdonly)
overhead

array([0.02530929, 0.04965271, 0.03858229, 0.60742771])

In [175]:
df = pd.read_csv("../../paper/communities_macrobenchmark.csv")
df = df[df["cell_type"]=="print_series"]
df = df.groupby(["nPts","condition"]).mean().reset_index()

pdonly = df[df["condition"]=="pandas"]
o1 = df[df["condition"]=="o1"]
o1o2 = df[df["condition"]=="o1o2"]
best = df[df["condition"]=="o1o2o3"]
worst = df[df["condition"]=="baseline"]

overhead = rawChange(best,pdonly)
overhead

array([0.0337895 , 0.03563925, 0.024035  , 0.0734725 ])

#### Non-Lux Operations

In [214]:
df = pd.read_csv("../../paper/airbnb_macrobenchmark.csv")
df = df[df["cell_type"]=="python"]
df = df.groupby(["nPts","condition"]).mean().reset_index()

pdonly = df[df["condition"]=="pandas"]
o1 = df[df["condition"]=="o1"]
o1o2 = df[df["condition"]=="o1o2"]
best = df[df["condition"]=="o1o2o3"]
worst = df[df["condition"]=="baseline"]

# rawChange(best,pdonly)
# pctChange(worst,best)
pctChange(worst,best)

array([  2.42555696,   4.61294096,  20.18075459, 105.86283076])

In [215]:
df = pd.read_csv("../../paper/communities_macrobenchmark.csv")
df = df[df["cell_type"]=="python"]
df = df.groupby(["nPts","condition"]).mean().reset_index()

pdonly = df[df["condition"]=="pandas"]
o1 = df[df["condition"]=="o1"]
o1o2 = df[df["condition"]=="o1o2"]
best = df[df["condition"]=="o1o2o3"]
worst = df[df["condition"]=="baseline"]

# rawChange(best,pdonly)
# pctChange(worst,best)
pctChange(worst,best)

array([ 227.84946663,  221.05188341, 1404.57006127,  688.8308745 ])